In [7]:


from nltk.ccg import chart, lexicon


lex = lexicon.parseLexicon('''
       :- S, NP, N, VP
    
        Det :: NP/N
        Pro :: NP
         Modal :: S\\NP/VP
    
         TV :: VP/NP
         DTV :: TV/NP
    
         the => Det
    
         that => Det
         a => Det
         the => Det
         that => NP
    
         I => Pro
         you => Pro
         we => Pro
    
         chef => N
         cake => N
         children => N
         dough => N
    
         will => Modal
         should => Modal
         might => Modal
         must => Modal
    
         and => var\\.,var/.,var
    
         to => VP[to]/VP
    
         without => (VP\\VP)/VP[ing]
    
         be => TV
         cook => TV
         eat => TV
    
         cooking => VP[ing]/NP
    
         give => DTV
    
         is => (S\\NP)/NP
        prefer => (S\\NP)/NP
    
        which => (N\\N)/(S/NP)
    
         persuade => (VP/VP[to])/NP
        ''')


<ipython-input-7-473d878506aa>:4: DeprecationWarning: 
  Function parseLexicon() has been deprecated.  Use fromstring()
  instead.
  lex = lexicon.parseLexicon('''


In [8]:
parser = chart.CCGChartParser(lex, chart.DefaultRuleSet)

In [9]:
for parse in parser.parse("you prefer that cake".split()):  # doctest: +SKIP
    chart.printCCGDerivation(parse)
    break

 you    prefer      that   cake
 NP   ((S\NP)/NP)  (NP/N)   N
                  -------------->
                        NP
     --------------------------->
               (S\NP)
--------------------------------<
               S


In [13]:
parser = chart.CCGChartParser(lex, chart.DefaultRuleSet)
for parse in parser.parse("I should give you the cake".split()):  # doctest: +SKIP
    chart.printCCGDerivation(parse)
    break

 I     should         give      you   the    cake
 NP  ((S\NP)/VP)  ((VP/NP)/NP)  NP   (NP/N)   N
                 ------------------->
                       (VP/NP)
                                    -------------->
                                          NP
                 --------------------------------->
                                VP
    ---------------------------------------------->
                        (S\NP)
--------------------------------------------------<
                        S


In [15]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [1]:
from nltk.parse import load_parser
from nltk.sem.drt import DrtParser
parser = load_parser('grammars/book_grammars/drt.fcfg', trace=1, logic_parser=DrtParser())
for tree in parser.parse('a dog barks'.split()):
     print(tree.label()['SEM'].simplify())

|. a  .dog .bark.|
|[----]    .    .| [0:1] 'a'
|.    [----]    .| [1:2] 'dog'
|.    .    [----]| [2:3] 'barks'
|[----]    .    .| [0:1] Det[NUM='sg', SEM=<\P Q.(([x],[]) + P(x) + Q(x))>] -> 'a' *
|[---->    .    .| [0:1] NP[NUM=?n, SEM=<?det(?nom)>] -> Det[NUM=?n, SEM=?det] * Nom[NUM=?n, SEM=?nom] {?det: <DrtLambdaExpression \P Q.(([x],[]) + P(x) + Q(x))>, ?n: 'sg'}
|.    [----]    .| [1:2] N[NUM='sg', SEM=<\x.([],[dog(x)])>] -> 'dog' *
|.    [----]    .| [1:2] Nom[NUM='sg', SEM=<\x.([],[dog(x)])>] -> N[NUM='sg', SEM=<\x.([],[dog(x)])>] *
|.    [---->    .| [1:2] Nom[NUM=?n, SEM=<?pp(?nom)>] -> N[NUM=?n, SEM=?nom] * PP[SEM=?pp] {?n: 'sg', ?nom: <DrtLambdaExpression \x.([],[dog(x)])>}
|[---------]    .| [0:2] NP[NUM='sg', SEM=<\Q.(([x],[dog(x)]) + Q(x))>] -> Det[NUM='sg', SEM=<\P Q.(([x],[]) + P(x) + Q(x))>] Nom[NUM='sg', SEM=<\x.([],[dog(x)])>] *
|[--------->    .| [0:2] S[SEM=<?subj(?vp)>] -> NP[NUM=?n, SEM=?subj] * VP[NUM=?n, SEM=?vp] {?n: 'sg', ?subj: <DrtLambdaExpression \Q.(([x],

In [1]:
from nltk.parse import load_parser
from nltk.sem.drt import DrtParser
parser = load_parser('grammar.fcfg', trace=1, logic_parser=DrtParser())
for tree in parser.parse('there is a table in the centre'.split()):
     print(tree.label()['SEM'].simplify())

|.t.i.a.t.i.t.c.|
|[-] . . . . . .| [0:1] 'there'
|. [-] . . . . .| [1:2] 'is'
|. . [-] . . . .| [2:3] 'a'
|. . . [-] . . .| [3:4] 'table'
|. . . . [-] . .| [4:5] 'in'
|. . . . . [-] .| [5:6] 'the'
|. . . . . . [-]| [6:7] 'centre'
|[-> . . . . . .| [0:1] VP[NUM=?n] -> 'there' * BE[NUM=?n] {}
|[-] . . . . . .| [0:1] THERE[] -> 'there' *
|. [-] . . . . .| [1:2] BE[NUM='sg'] -> 'is' *
|[---] . . . . .| [0:2] VP[NUM='sg'] -> 'there' BE[NUM='sg'] *
|[---> . . . . .| [0:2] S[SEM=?np] -> VP[NUM=?n] * NPP[NUM=?n, SEM=?np] {?n: 'sg'}
|. . [-] . . . .| [2:3] Det[NUM='sg', SEM=<\P Q.(([x],[]) + P(x) + Q(x))>] -> 'a' *
|. . [-> . . . .| [2:3] NP[NUM=?n, SEM=<?det(?np)>] -> Det[NUM=?n, SEM=?det] * N[NUM=?n, SEM=?np] {?det: <DrtLambdaExpression \P Q.(([x],[]) + P(x) + Q(x))>, ?n: 'sg'}
|. . . [-] . . .| [3:4] N[NUM='sg', SEM=<\x.([],[table(x)])>] -> 'table' *
|. . [---] . . .| [2:4] NP[NUM='sg', SEM=<\Q.(([x],[table(x)]) + Q(x))>] -> Det[NUM='sg', SEM=<\P Q.(([x],[]) + P(x) + Q(x))>] N[NUM='sg', SEM

In [ ]:
for tree in parser.parse('The table is 1.8m wide and 0.8m long'.split()):
     print(tree.label()['SEM'].simplify())

In [129]:
class Vector3D():
    
    def __init__(self, x, y ,z):
        self.x = x
        self.y = y
        self.z = z

    def __repr__(self):
        return f"Vector3D({self.x}, {self.y}, {self.z})"
    
class Distribution():
    
    distribution_name = ""
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f"{self.distribution_name}({self.x}, {self.y})"
    

class Normal(Distribution):
    
    distribution_name = "Normal"
    
# class Discrete(Distribution):
    
#     distribution_name = "Discrete"
    
class math_expr():
    
    def __repr__(self):
        return ""

class Max(math_expr):
    
    name = "max"
    
    def __init__(self, numbers):
        self.numbers = numbers
        
    def __repr__(self):
        return f"{self.name}({self.numbers})"
    
class Min(math_expr):
    
    name = "min"
    
    def __init__(self, numbers):
        self.numbers = numbers
        
    def __repr__(self):
        return f"{self.name}({self.numbers})"
    
class Abs(math_expr):
    
    name = "abs"
    
    def __init__(self, numbers):
        self.numbers = numbers
        
    def __repr__(self):
        return f"{self.name}({self.numbers})"
    

class Plus(math_expr):
    
    name = "+"
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f"{self.x} {self.name} {self.y}"
    
class Times(math_expr):
    
    name = "*"
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f"{self.x} {self.name} {self.y}"
    
class DistanceFrom(math_expr):
    
    name = "distance from"
    
    def __init__(self, vector1, vector2):
        self.vector1 = vector1
        self.vector2 = vector2
        
    def __repr__(self):
        return f"distance from {self.vector1} to {self.vector2}"
    
class RelativeTo():
    
    def __init__(self, vector1, vector2):
        self.vector1 = vector1
        self.vector2 = vector2
        
    def __repr__(self):
        return f"{self.vector1} relative to {self.vector2}"
    
class BooleanExpr():
    
    name = ""
    
    def __init__(self, expr1, expr2):
        self.expr1 = expr1
        self.expr2 = expr2
        
    def __repr__(self):
        return f"{self.expr1} {self.name} {self.expr2}"

class NotBool(BooleanExpr):
    
    name = "not"
    
    def __init__(self, expr):
        self.expr = expr
        
    def __repr__(self):
        return f"not {self.expr}"
    
class AndBool(BooleanExpr):
    
    name = "and"
    
    def __init__(self, expr1, expr2):
        self.expr1 = expr1
        self.expr2 = expr2
        
    def __repr__(self):
        return f"{self.expr1} and {self.expr2}"
    
class OrBool(BooleanExpr):
    
    name = "or"
    
    def __init__(self, expr1, expr2):
        self.expr1 = expr1
        self.expr2 = expr2
        
    def __repr__(self):
        return f"{self.expr1} or {self.expr2}"
    
class EQ(BooleanExpr):
    name = "=="

class NEQ(BooleanExpr):
    name = "!="

class LT(BooleanExpr):
    name = "<"

class GT(BooleanExpr):
    name = ">"

class GEQ(BooleanExpr):
    name = ">="

class LEQ(BooleanExpr):
    name = "<="

class IsIn(BooleanExpr):
    name = "is in"
    
    

class CuboidRegion():
    
    def __init__(self, vector1, vector2, vector3):
        self.vector1 = vector1
        self.vector2 = vector2
        self.vector3 = vector3
        
    def __repr__(self):
        return f"CuboidRegion({self.vector1}, {self.vector2}, {self.vector3})"

    
class Entity():
    
    def __init__(self, type_, name):
        self.type = type_
        self.name = name
        self.constraints = []
        
    def add_constraint(self, constraint):
        self.constraints.append(constraint)
        
    def __repr__(self):
        constraint_repr = ', '.join([con.__repr__() for con in self.constraints])
        
        return f"{self.name} = {self.type}" + constraint_repr
    


    
    
class WorldModel():
    
    def __init__(self):
        pass

    

    
class InterpretationSystem():
    
    def __init__(self):
        self.dialogue_history = []
        self.dialogue_referents = []
        self.parser = load_parser('grammar.fcfg', trace=0, logic_parser=DrtParser())
        
        
    def read_sentence(self, sentence: str):
        for tree in self.parser.parse(sentence.split()):
             sem = tree.label()['SEM'].simplify()
        
        return sem
    
    def read_DRS(self, drs: DRS):
        
        refs = [r.name for r in drs.refs]
        preds = [parse_cond(cond) for cond in drs.conds]
        return DRSState(preds, refs)
            

            
                
def parse_cond(cond):
    if isinstance(cond, DrtApplicationExpression):
        pred = cond.pred.variable.name
#         print(pred)
        args = [parse_cond(x) for x in cond.args]
#         print(args)

        return Predicate(pred, args)

#         if pred in ['table', 'cup']:
            
#             obj = (pred, args)
#             return obj
            
#         elif pred in ['vector']:
#             vector = Vector3D(*args)
#             return vector
#         elif pred in ['on']:
#             return (pred, args)
        
    if isinstance(cond, DrtIndividualVariableExpression) or isinstance(cond, DrtConstantExpression):
        return cond.variable.name

In [130]:

class Predicate(object):

    def __init__(self, name, args, op=None):
        self.name = name
        self.args = args
        self.valency = not(op == 'not')
        self.op = op

    def negate(self):
        op = 'not' if self.valency is True else None
        return Predicate(self.name, self.args, op)

    def __str__(self):
        return f"{self.name}({','.join(map(str, self.args))})"

    def __repr__(self):
        return self.__str__()

    def __eq__(self, other):
        return self.name == other.name and all([arg1 == arg2 for arg1, arg2 in zip(self.args, other.args)]) and len(self.args) == len(other.args)

    def __ne__(self, other):
        return not(self.__eq__(other))

    def to_dexpr(self):
        
        return DrtExpression.fromstring(self.__str__())
        

    
class DRSState(object):
    def __init__(self, predicates, referents):
        self.referents = referents
        self.predicates = predicates

    def get_predicates(self, arg):
        return [pred for pred in self.predicates if arg in pred.args]

    def _predicate_holds(self, predicate):
        return predicate in self.predicates

    def predicate_holds(self, predicate_name, args):
        predicate = Predicate(predicate_name, args)

    def __eq__(self, other):
        for predicate in self.predicates:
            if predicate not in other.predicates:
                return False
        for predicate in other.predicates:
            if predicate not in self.predicates:
                return False
        return True

    def asDRS(self):
        dexpr = [pred.to_dexpr() for pred in self.predicates]
        return DRS(self.referents, dexpr)
    
    def print(self):
        drs = self.asDRS()
        drs.pretty_print()


In [131]:
preds = [Predicate('table', ['x']), Predicate('on', ['x', Predicate('vector', [0, 0, 0])])]
d = DRSState(preds, ['x'])

In [132]:
d.print()

 _____________________ 
| x                   |
|---------------------|
| table(x)            |
| on(x,vector(0,0,0)) |
|_____________________|


In [133]:
from nltk.sem.drt import *

In [134]:
s = InterpretationSystem()
sem = s.read_sentence("there is a table in the centre")
s.read_DRS(sem).print()


 _____________________ 
| x                   |
|---------------------|
| table(x)            |
| on(x,vector(0,0,0)) |
|_____________________|


In [114]:
sem.pretty_print()

 _____________________ 
| x                   |
|---------------------|
| table(x)            |
| on(x,vector(0,0,0)) |
|_____________________|


In [108]:
sem.predicates()
fol = sem.fol()

In [111]:
sem.refs[0].name

'x'

In [89]:
t.args[1].args[0].variable.name

'0'

In [49]:
fol_sem = sem.fol()

In [50]:
and_expr = fol_sem.term

In [51]:
f = and_expr.first

In [52]:
s= and_expr.second

In [53]:
s.args

[<DrtIndividualVariableExpression x>, <ApplicationExpression vector(0,0,0)>]